# Waveform Tutorial

March 6, 2019    
Edited May 13, 2019

----

## Extracting Waveforms

When a certain waveform quantity (e.g. $h$, $\Psi_4$, etc.) is computed in the Spectral Einstein Code (SpEC), it is decomposed in terms of the spin-weighted spherical harmonics (SWSHs) ${}_{s}Y_{\ell}^{m}$ with the appropriate spin-weight $s$ for the quantity. For example,

$$ h(r,t,\theta,\phi) = \sum_{\ell=2} \sum_{m=-\ell}^{\ell} h^{(\ell,m)}(r,t)\; {}_{-2}Y_{\ell}^{m}(\theta,\phi) \hspace{3cm} (1)$$

The mode weights of this decomposition can be extracted from the simulation and can be used with the equation above to get $h(r,t,\theta,\phi)$. Since the strain $h$ and the Weyl scalar $\Psi_4$ fall off predominantly as $r^{-1}$, we multiply the values by the extraction radius before writing to disk, which is why the file names are `rh*.h5` and `rPsi4*.h5`. So in our example, we would write $rh^{(\ell,m)}(r,t)$ to disk as an HDF5 file for all timesteps $t$ in our simulation and for the specific values of the radius $r$ at which we measured this quantity.


## Extrapolation

We expect that the waveform at asymptotic null infinity has the following fall-off behavior:
$$ \lim_{r\rightarrow\infty} h \sim \frac{1}{r} \hspace{3cm} (2)$$
However, it is evident that our waveforms extracted at finite radii have higher order terms (e.g. $r^{-2}$, $r^{-3}$, etc.) that should not be present in a waveform observed at null infinity. We therefore apply an extrapolation procedure to our finite radius waveforms in order to have a more reasonable final waveform. We also apply a correction for center-of-mass drift in the simulation. To allow for comparison, we provide the following data for each simulation:

* `*_Asymptotic_GeometricUnits_CoM.h5` -- waveforms that have been extrapolated and center-of-mass corrected
* `*_Asymptotic_GeometricUnits.h5` -- waveforms that have been extrapolated
* `*_FiniteRadii_CodeUnits.h5` -- raw finite radius waveforms

In our catalog as of March 6, 2019, **only** the waveforms ending with `*_Asymptotic_GeometricUnits_CoM.h5` should be used for analysis. If more corrections have been added to the waveforms since then, please use the more up-to-date versions and update this tutorial. These waveforms also inlcude a factor of $M$ as specified in the name of the file in order to render the data dimensionless ($G=c=1$):
$$ r M^{-1} h \hspace{1cm} r M \Psi_4 $$

For a discussion of the extrapolation procedure, please see:
[M. Boyle and A. H. Mroué, *Phys. Rev.* **D80**, 124045 (2009)](https://arxiv.org/abs/0905.3177)  
For a discussion of other corrections applied to the waveforms, please see:
[M. Boyle, *Phys. Rev.* **D93**, 084031 (2016)](https://arxiv.org/abs/1509.00862)

Note that the finite radius waveforms use coordinate radius values and the extrapolated waveforms use areal radius:
$$ r = \sqrt{\frac{1}{4\pi} \int_{S^2} |g|\, d\Omega}$$
where $|g|$ is the determinant of the metric, and the surface $S^2$ is the surface of constant coordinate radius where the values of the waveform computed in the simulation.

---

The Python module [h5py](https://www.h5py.org/]) provides convenient tools for working with HDF5 files, which contain an internal directory structure that nicely organizes the data. Let's start by opening up an `rh_Asymptotic_GeometricUnits_CoM.h5` file. To follow this notebook, click [here](https://zenodo.org/record/1215624/files/SXS:BBH:1355/Lev1/rhOverM_Asymptotic_GeometricUnits_CoM.h5?download=1) to download a short waveform of the gravitational radiation $rh$ from simulation SXS:BBH:1355 in our catalog. This waveform has the lowest resolution data (Lev1) for an eccentric equal-mass non-spinning binary black hole simulation. 

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# Please uncomment the line below and fill in the correct path
rh_file = # "/path/to/rhOverM_Asymptotic_GeometricUnits_CoM.h5"

# Open the HDF5 file in read-only mode:
rh_data = h5py.File(rh_file,'r')

The second argument `'r'` that we supplied in `h5py.File` sets the file to read-only mode. The following table lists the other arguments that can be supplied instead:

| Argument    | Description                                      |
|  :----:     | :----:                                           |
| `r`         | Readonly, file must exist                        |
| `r+`	      | Read/write, file must exist                      |
| `w-` or `x` | Create file, fail if exists                      |
| `a`	      | Read/write if exists, create otherwise (default) |



We navigate through the HDF5 directory structure by specifying the path as a key to `rh_data`, which is essentially a dictionary. HDF5 directories are known as "groups", so the root group is:

`rh_data['/']`

If you instead just call:

`rh_data`

this is assumed to be the root group and is equivalent to the `rh_data['/']` call.

In [ ]:
# Let's see what groups we have in the root group:
for group in sorted(rh_data): 
    print(group)
    
# Can also just do:
# > print(sorted(rh_data))

We should find three groups called `Extrapolated_N*.dir` that holds waveforms extrapolated to different orders, and one group called `OutermostExtraction.dir` that holds waveforms extracted from the outermost extraction radius in the simulation. Accuracy of extrapolation to future null infinity can be assessed by comparing different extrapolation orders. The 'best' extrapolation order to use depends on the simulation and what the waveform is being used for. Higher order tends to do better in the inspiral but worse in the ringdown; it is likely that the `OutermostExtraction` data is better than extrapolated data during ringdown.

Let's now see what we have in one of these groups.

In [ ]:
for group in sorted(rh_data['Extrapolated_N4.dir']): 
    print(group)

We now see that this group is full of datasets. There is a file `History.txt` that records any changes to the data in post-processing (i.e. extrapolation, correction for center-of-mass drift, etc.), and the rest are datasets labelled `Y_l*_m*.dat`. These are the datasets that hold the complex mode weights for $rh$. The spin-weight of the spherical harmonic decomposition is assumed! When computing $h$ from the mode weights, be sure you use the spin-weighted spherical harmonics and not just the usual (spin-weight = 0) spherical harmonics. In other words, these are ***not*** the mode weights for $Y_{\ell}^{m}$ but for ${}_{-2}Y_{\ell}^{m}$.

The dominant modes for the strain are the $(\ell=2,m=\pm 2)$ modes so let's take a look at one of those datasets.

In [ ]:
rh_data['Extrapolated_N4.dir/Y_l2_m2.dat'].shape

# For the m=-2 mode:
# rh_data['R0545.dir/Y_l2_m-2.dat'].shape

The first column of the dataset is the retarted time, the second column is the real part of $rh^{(\ell,m)}$, and the third column is the imaginary part of $rh^{(\ell,m)}$. We see from above that we have 12855 timesteps in this dataset. For convenience, let's build a 1D array holding the complex waveform instead of having to access different columns of the HDF5 file.

In [ ]:
# Let's take the l=m=2 mode 
idx = 'Extrapolated_N4.dir/Y_l2_m2.dat'
ret_time = rh_data[idx][:,0]
rh = rh_data[idx][:,1] + 1j*rh_data[idx][:,2]

# To plot the waveform interactively, uncomment the following line:
# %matplotlib qt
plt.plot(ret_time, np.real(rh), label='Real')
plt.plot(ret_time, np.imag(rh), label='Imag')
plt.plot(ret_time,  np.abs(rh), label='Abs')
plt.title('Extrapolated Waveform, N=4')
plt.xlabel('$(t_{corr} - r^{*})/M$')
plt.ylabel('$rh^{(2,2)}$')
plt.legend()
plt.show()

Often, the plot above is colloquially referred to as a plot of $rh$, but remember that it is *actually* a plot of just the $\ell=m=2$ mode weight of $rh$!

It might also be useful to plot $rh^{(\ell,m)}$ for not just the leading order $(2,2)$ mode but for other modes as well.

In [ ]:
for lm in [(2,2),(4,4),(3,2),(6,6),(5,4),(8,8)]:
    l = str(lm[0])
    m = str(lm[1])
    idx = 'Extrapolated_N4.dir/Y_l'+l+'_m'+m+'.dat'
    ret_time = rh_data[idx][:,0] - 545
    rh = rh_data[idx][:,1] + 1j*rh_data[idx][:,2] 
    
    plt.semilogy(ret_time, np.abs(rh), label='('+l+','+m+')')
    
plt.title('Extrapolated Waveform, N=4')
plt.xlabel('$(t_{corr} - r^{*})/M$')
plt.ylabel('$rh^{(\ell,m)}$')
plt.legend()
plt.show()

In [ ]:
# Remember to close the file when you are done
rh_data.close()

---

## Getting a GW Quantity at a Point on the Sphere

Before getting started, you will need the following Python modules:
* [quaternion](https://github.com/moble/quaternion)
* [spherical_functions](https://github.com/moble/spherical_functions)

To compute the actual value of $h$ and not just the mode weights, we need to perform the summation in Eq. (1), reproduced here for convenience:
$$ h(r,t,\theta,\phi) = \frac{1}{r} \sum_{\ell} \sum_{m=-\ell}^{\ell} h^{(\ell,m)}(r,t)\; {}_{-2}Y_{\ell}^{m}(\theta,\phi)$$

Unlike the mode weights, the value of $rh$ is a function of $(\theta,\phi)$ so we need to specify a point on the sphere at which to define the SWSHs. Let's define two functions to make our life easier first:

In [ ]:
def swsh(s, lm_list, theta, phi, psi=0):
    import spherical_functions as sf
    import quaternion as qt
    """
    Given a list of modes as (l,m) pairs, returns a list of values of those
    spin-weighted spherical harmonic modes at a point.

    s = spin-weight
    lm_list = [(l,m)] or [(l1,m1),(l2,m2),...]
    theta = polar angle
    phi = azimuthal angle 
    psi = frame orientation
    """
    return sf.SWSH(qt.from_spherical_coords(theta, phi), s, lm_list)*np.exp(1j*s*psi)      


def get_at_point(waveform_modes, lm_list, theta, phi, psi=0, s=-2):
    """
    Computes the value of a gravitational wave quantity at a point on the sphere.
    
    waveform_modes = array where each element is a 1D complex waveform of a single (l,m) mode
    lm_list = a list of all the (l,m) pairs in waveform_modes
    theta = polar angle
    phi = azimuthal angle 
    psi = frame orientation
    s = spin-weight   
    """
    waveform_at_point = np.empty(waveform_modes.shape[1], dtype=complex)
    # For each timestep, compute Eq. (1)
    for t in range(waveform_modes.shape[1]):
        waveform_at_point[t] = np.sum(
            waveform_modes[:, t]*swsh(s, lm_list, theta, phi, psi))
    return waveform_at_point

Now we can load the extrapolated data from before and pre-compute:
* a list of the $(l,m)$ pairs that we want to include in our summation (here we are going to use all of them)
* an array where each element is a 1D complex mode weight waveform

In [ ]:
rh_file = # "/path/to/rhOverM_Asymptotic_GeometricUnits_CoM.h5"                                                                 
rh_data = h5py.File(rh_file,'r')

# Get a list of all the data's (l,m) pairs
lm_list = []
mode_strs = sorted(rh_data[sorted(rh_data)[0]])
for mode in mode_strs:
    if mode.startswith('Y'):
        # The l value is the 4th char in the string 'Y_l*_m*.dat'
        # but the m value could have a minus sign so we 
        # make sure we include that if it's present
        if mode[6] == '-':
            lm_list.append((int(mode[3]), int(mode[6:8])))
        else:
            lm_list.append((int(mode[3]), int(mode[6])))

# Alternatively, you could also define your own list of lm pairs 
# if you only wanted the l=2 modes in the summation for example:
# lm_list = [(2,-2),(2,-1),(2,0),(2,1),(2,2)]

waveform_modes = []
for lm_pair in lm_list:
    idx = 'Extrapolated_N4.dir/Y_l'+str(lm_pair[0])+'_m'+str(lm_pair[1])+'.dat'
    waveform_modes.append(rh_data[idx][:,1] + 1j*rh_data[idx][:,2])
waveform_modes = np.array(waveform_modes)

ret_time = rh_data[idx][:,0]

With all this in hand, we can make a plot of the $rh_+$ waveform at a few points on the sphere:

In [ ]:
for theta in [0,0.25,0.4,0.5]:
    plt.plot(ret_time, np.real(get_at_point(waveform_modes, lm_list, theta*np.pi, 0.0*np.pi)), label='$\theta='+str(theta)+'\pi$')
plt.title('$rh_{+}$ Extrapolated Waveform, N=4')
plt.xlabel('$(t_{corr} - r^{*})/M$')
plt.ylabel('$rh$')
plt.legend()
plt.show()

for theta in [0,0.25,0.4,0.5]:
    plt.plot(ret_time, np.imag(get_at_point(waveform_modes, lm_list, theta*np.pi, 0.0*np.pi)), label='$\theta='+str(theta)+'\pi$')
plt.title('$rh_x$ Extrapolated Waveform, N=4')
plt.xlabel('$(t_{corr} - r^{*})/M$')
plt.ylabel('$rh$')
plt.legend()
plt.show()

In [ ]:
rh_data.close()